In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
#from flame_net.PFNO_Nd import PFNO_Nd
#from flame_net.FourierOp_Nd import FourierOp_Nd
#from flame_net.DeepONet_1d import DeepONet_1d
#from flame_net.ConvPDE_Nd import ConvPDE_Nd
from flame_net.lib_uti import Cdata_sys, count_learnable_params, lib_Model,lib_DataGen,LpLoss,lib_ModelTrain
from timeit import default_timer

#-----------------------
device = torch.device('cuda') # if torch.cuda.is_available() else 'cpu')


#-----------------------
nDIM = 1
#data_sys = Cdata_sys('siva',[0.025, 0.075, 0.2], 4,'MS_1stEuler')

list_para=[ [10,0],[10,0.25],[10,0.5], [10,0.75], [10,1],
            [25,0],[25,0.25],[25,0.5], [25,0.75], [25,1],
            [40,0],[40,0.25],[40,0.5], [40,0.75], [40,1]  ]

data_sys = Cdata_sys('MKS_RK4', list_para,  num_PDEParameters=2 )

#data_sys = Cdata_sys('cfd',['L1536_rho8'])
params = lib_Model.set_default_params(data_sys,nDIM)


params['T_in' ] = 1
params['T_out'] = 20
params['data_channel'] = 1


params['data:nStep']  = 1
params['data:nStepSkip']=1
params['Nx']            = 256 # 512 # 128
#-----
params['train:batch_size'] = 1000
params['train:learning_rate'] = 0.0025
#---------------
#params['optimizer_method']=torch.optim.AdamW
#-----------
params['train:gradient_clip'] = 50

params['tensorboard_logdir_prefix'] = ''
params['model_name_prefix'] = ''


dataset_train, dataset_test                        = lib_DataGen.DataGen(data_sys,params)

----- params for DataGen -----
nDIM,T_in,T_out,Nx,nStep,nStepSkip,data_channel, data_sys.sys_name,data_sys.para_name(), data_sys.list_para)   #, data_sys.list_cfdfilename
------------------------------
siva_sys_name= MKS_RK4
N= 256 ,dt_Output= 0.15 ,self.list_para = [[10, 0], [10, 0.25], [10, 0.5], [10, 0.75], [10, 1], [25, 0], [25, 0.25], [25, 0.5], [25, 0.75], [25, 1], [40, 0], [40, 0.25], [40, 0.5], [40, 0.75], [40, 1]] ,[(1, 125001, [40, 0], 'rand_simple'), (250, 501, [40, 0], 'rand_simple'), (1, 125001, [40, 0.25], 'rand_simple'), (250, 501, [40, 0.25], 'rand_simple'), (1, 125001, [40, 0.5], 'rand_simple'), (250, 501, [40, 0.5], 'rand_simple'), (1, 125001, [40, 0.75], 'rand_simple'), (250, 501, [40, 0.75], 'rand_simple'), (1, 125001, [40, 1], 'rand_simple'), (250, 501, [40, 1], 'rand_simple'), (1, 125001, [25, 0], 'rand_simple'), (250, 501, [25, 0], 'rand_simple'), (1, 125001, [25, 0.25], 'rand_simple'), (250, 501, [25, 0.25], 'rand_simple'), (1, 125001, [25, 0.5], 'rand_simple'),

In [2]:
#%matplotlib inline
#plt.figure(figsize=[4,4])
#plt.contourf(  train_disp[46130,:,:]  )
#plt.contourf(train_disp[-1,:,:])
#plt.plot(train_PDEpara)
#plt.plot(train_disp[-1,:,-1])






In [3]:
model_name = 'conv'
if 'fno' == model_name :
    params['fourier:modes_fourier' ] =  128  # 64 # 128 # 128 # 64
    params['fourier:width' ] =  30           # 25 # 20
    params['fourier:depth' ] =  4
    params['fourier:method_WeightSharing']  = True #
    params['fourier:method_SkipConnection'] = False
    params['fourier:method_ParaEmbedding']  = True # False # True #  3 # False # 2 #False # 2 or True
    params['fourier:PDEPara_mode_level']    = [5] # [7] #   [3,5]# [7]      #[2,5] # 7 # 5 # None # 5
    params['PDEPara_fc_class'] = 'OM'

    # params['fourier:method_SteadySolution'] =  True

elif 'conv' == model_name:

    params['conv:method_types_conv'] = 'inception_less'
    params['conv:en1_channels' ] = [ [16,32],[32,32],[64,64],[128],[128],[64],[32]] # [ [16],[32,32],[64,64],[128],[128],[64],[32]]
    params['conv:PDEPara_depth'] = 6                                                # 6
    params['conv:method_BatchNorm' ] = 256
    params['conv:method_ParaEmbedding']= False # True
    params['conv:method_nonlinear' ] = 'all_delay'
    params['conv:method_ParaEmbedding' ] = True
    params['conv:PDEPara_PathNum' ] = 2

    params['model_name_prefix'] = ''


# params['PDEPara_OutValueRange']=0.3
# params['method_outputTanh' ] = False        # True
# params['train:epochs'] = 1000
# params['train:checkpoint_resume'] = '_best.pt' # None     # '_best.pt'

In [4]:
model_name_detail = lib_Model.get_model_name_detail(model_name,data_sys,params)
model = lib_Model.build_model(model_name_detail,params)

p2Conv_Lpi_rho10_0_40_1Ed6_inception_less_nonlinearall_delay_pathnum2_ln_o20
ConvPDE_Nd: en_channels =  [[1, 16, 32], [32, 32, 32], [32, 64, 64], [64, 128], [128, 128], [128, 64], [64, 32]]
ConvPDE_Nd: de_channels =  [[32, 32], [64, 64], [128, 128], [256, 128], [256, 64], [128, 32], [64, 32], [64, 1]]
ConvPDE_Nd: en_types =  [['c', 'i'], ['c', 'i'], ['c', 'i'], ['i'], ['c'], ['c'], ['c']]
ConvPDE_Nd: de_types =  [['c'], ['c'], ['c'], ['c'], ['c'], ['c'], ['i'], ['c']]
count_learnable_params= 514839


In [5]:
model(torch.rand(1,256,1).to(device), torch.rand(1,2).to(device)).shape
#model.en_delay_nonlinear_norm
#model.en_conv_PDEPara

torch.Size([1, 256, 1])

In [5]:

# def tensorboard_fig1d_monitor( va_Pair_pick, vaPDEPara_Pair_pick, model, device='cuda'):
#     # only handle 1-D
#     nSize = va_Pair_pick.shape[0]
#     #fig, axs = plt.subplots( nSize,1, figsize=(15, 15) )
#     fig, ax = plt.subplots( 1,1, figsize=(9, 8) )
#     model.eval()
#     with torch.no_grad():
#         va_pred = model( va_Pair_pick[:,:,0:1].to(device), vaPDEPara_Pair_pick.to(device)  ).detach().to('cpu')
#     line1color = 'kbrg'
#     line2color = 'kbrg'
#     # if nSize >1:
#     for i in range(nSize):
#         #axs[i].plot( va_Pair_pick[icde,:,1]/20 , 'c')

#         #axs[i].plot( va_Pair_pick[i,:,1]-va_Pair_pick[i,:,0],'k--')
#         #axs[i].plot( va_pred[i,:,0] - va_Pair_pick[i,:,0], 'r-' )

#         ax.plot( va_Pair_pick[i,:,1]-va_Pair_pick[i,:,0], line1color[i]+'--')
#         ax.plot( va_pred[i,:,0]    - va_Pair_pick[i,:,0], line2color[i]+'-' )
#     # else:
#     #     i = 0
#     #     #axs.plot( va_Pair_pick[i,:,1]/20 , 'c')
#     #     axs.plot( va_Pair_pick[i,:,1]-va_Pair_pick[i,:,0],'k--')
#     #     axs.plot( va_pred[i,:,0] - va_Pair_pick[i,:,1], 'r-' )
#     fig.tight_layout()
#     return fig

In [6]:
%matplotlib inline
from flame_net.lib_uti import tensorboard_fig1d_monitor 

n_list = [int(len(dataset_test)//3*0.9), int(len(dataset_test)//3*1.3), int(len(dataset_test)//3*2.1)]
disp_peek = torch.stack( [dataset_test[n][0] for n in n_list ] )
para_peek = torch.stack( [dataset_test[n][1] for n in n_list ] )

def tensorboard_callback(ep, device):
    #n1,n2,n3 = 351,1050, -522
    if ep%50==0:
#         return tensorboard_fig1d_monitor( torch.stack( (test_disp[n1],test_disp[n2], test_disp[n3]) ) , 
#                                           torch.stack( (test_PDEpara[n1],test_PDEpara[n2], test_PDEpara[n3]) ),
#                                           model, device) 
        return tensorboard_fig1d_monitor( disp_peek, para_peek,   model, device, params['T_in'], params['option_nOutputStep'] ) 

    else: 
        return None
#fig = tensorboard_callback()
params['tensorboard_fig1d']= tensorboard_callback

In [7]:
#torch.stack( (test_PDEpara[1],test_PDEpara[2], test_PDEpara[3]) )
#train_PDEpara.shape

In [8]:
#tensorboard_callback(0)

In [9]:
params['train:batch_size'] =1000
#lib_ModelTrain.Train(train_disp, test_disp,train_PDEpara,test_PDEpara,model,model_name_detail,device,params )
lib_ModelTrain.Train(dataset_train, dataset_test                      ,model,model_name_detail,params )


batch_size= 1000
ntrain= 179970  ,ntest= 17970


2024-04-05 18:47:23.844585: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


....................................................................................................................................................................................
ep t[s] train_l2 test_l2 
0, 250.28, 0.72577, 0.46158
..........

KeyboardInterrupt: 